In [ ]:
import pandas as pd
import pydub
import os
from IPython.display import clear_output
from uitiles import uitiles

# Check .tsv files

from Github for Mozile common voice https://github.com/common-voice/cv-dataset

"We use the Mozilla Corpora Creator tool to parse through metadata to generate test, train, and dev sets. The Corpora Creator eliminates duplication in clips and maximized for speaker diversity.

Each test/train/dev set is generated non-deterministically, meaning that they will vary from release to release even for minor updates. This is to avoid reproducing and perpetuating any demographic skews in each subsequent set."

We will use train.tsv, test.tsv and dev.tsv as this files generated from validated 

Columns description 

client_id - hashed UUID of a given user
path - relative path of the audio file
text - supposed transcription of the audio
up_votes - number of people who said audio matches the text
down_votes - number of people who said audio does not match text
age - age of the speaker*
gender - gender of the speaker*
accent - accent of the speaker*
segment - if sentence belongs to a custom dataset segment, it will be listed here

In [5]:
# class uitiles:
#     DataSet_DF_columns=["filename","speaker","language","length","gender","accent","datasetname"]

    
#     def Generate_File_Name(Language :str, Gender:str, I_Speaker:int, I_Clip:int,datasetname: str,Extention: str):
#         return Language + "-" + Gender + "-" + str(I_Speaker) + "-" + str(I_Clip) + "-" + datasetname + "." + Extention

#     def Save_To_CSV(PathToDF, df):
#         if os.path.exists(PathToDF):
#             df.to_csv(PathToDF, mode='a', header=False)
#         else :
#             df.to_csv(PathToDF)


In [13]:
class ReadMozilaDataset:
    def __init__(self,ReadTSVPath,ReadClipsPath,SavePath,DataSetName):
        """
        Read mb3 files from Mozila common voice and convert them to WAV

        args :
            ReadTSVPath(str) : path to Folder that contains Train.tsv, Test.tsv, and dev.tsv files
            ReadClipsPath(str) : path to Folder that contains.mb3 clips
            SavePath(str) : path to Folder to save converted files in it
            DataSetName(str) : Will used to build new name for converted file
        """
        self.ReadTSVPath=ReadTSVPath
        self.ReadClipsPath=ReadClipsPath
        self.SavePath=SavePath
        self.DataSetName=DataSetName
        self.DataInitialDF= self.ReadTSVFiles()

    def ReadTSVFiles(self):
        """Read Train, Test, and dev files and Concat them in one dataFrame
        
        args :
            FolderPath(str) : path to Folder that contains Train.tsv, Test.tsv, and dev.tsv files
        
        return :
             concated DataFrame with data from Train, Test, and dev files

        """
        train = pd.read_csv(self.ReadTSVPath+'/train.tsv', sep='\t')
        test = pd.read_csv(self.ReadTSVPath+'/test.tsv', sep='\t')
        dev = pd.read_csv(self.ReadTSVPath+'/dev.tsv', sep='\t')
        
        return pd.concat([train, test ,dev], ignore_index=True)
         

    def Convert(self, SaveToDF=False,PathToSave=""):
        """ 
        Using Concated DataFrame to Read mb3 files and convert it and return new DF with converted files

        args:
            SaveToDF : To save to converted files metadate to .csv file or add to exist .csv file, defulat=False 

            PathToSave : Path to save csv file or to add to exist csv fill NOTE:You must enter path with filename.csv example ../test.csv
        """

        Speakers =self.DataInitialDF["client_id"].unique()
        ConvertedFiles = []
        for I_Speaker, Speaker in enumerate(Speakers):
            SpeakerClips = self.DataInitialDF[self.DataInitialDF["client_id"]==Speaker]
            print(f"I_speaker  : {I_Speaker} / {len(Speakers)}")

            for index, clip in SpeakerClips.iterrows():
                lang = clip["locale"]   
                gender =  str(clip["gender"])[0]  if str(clip["gender"])[0] !='o' else 'n'
                accent =  str(clip["accents"])
                NewName= uitiles.Generate_File_Name(lang, gender, I_Speaker, index, self.DataSetName,"wav")  
                try:
                    sound = pydub.AudioSegment.from_mp3(f"{self.ReadClipsPath}/{clip['path']}")
                    # sound.export(f"{self.SavePath}/{NewName}", format="wav") 
                    ConvertedFiles.append([NewName,Speaker,lang,sound.duration_seconds,gender,accent,self.DataSetName])
                except:
                    print(f"Error while converting fille {clip['path']}")
                    continue
            clear_output(wait=True)
            
        NewDF = pd.DataFrame(ConvertedFiles,columns=uitiles.DataSet_DF_columns)           
        if SaveToDF:
            uitiles.Save_To_CSV(PathToSave,NewDF)


        return NewDF

        

In [7]:
# ar = ReadMozilaDataset("../../DataSets/ExalVoxForge/mozila-ar","../../DataSets/ExalVoxForge/mozila-ar/clips","../../DataSets/FinalDataSet/clips","CV")
# ar.Convert(True,"../../DataSets/FinalDataSet/FinalDataSet.csv")


,filename,speaker,language,length,gender,accent,datasetname
0,ar-n-0-0-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.060,n,nan,CV
1,ar-n-0-1-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.176,n,nan,CV
2,ar-n-0-2-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,2.628,n,nan,CV
3,ar-n-0-3-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.168,n,nan,CV
4,ar-n-0-4-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.716,n,nan,CV
...,...,...,...,...,...,...,...
48862,ar-n-928-48862-CV.wav,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,ar,4.032,n,nan,CV
48863,ar-n-928-48863-CV.wav,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,ar,5.976,n,nan,CV
48864,ar-n-928-48864-CV.wav,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,ar,3.432,n,nan,CV
48865,ar-n-928-48865-CV.wav,61ca692e975b8e4882cdfcb40d756ab2b8e9eeeb6b9fb9...,ar,2.904,n,nan,CV


In [8]:
# po = ReadMozilaDataset("../../DataSets/ExalVoxForge/po","../../DataSets/ExalVoxForge/po/clips","../../DataSets/FinalDataSet/clips","CV")
# po.Convert(True,"../../DataSets/FinalDataSet/FinalDataSet.csv")



,filename,speaker,language,length,gender,accent,datasetname
0,pt-n-0-0-CV.wav,d4c57a15a1126889ae40be4e254341e394d40a5723de32...,pt,6.360,n,nan,CV
1,pt-n-0-1-CV.wav,d4c57a15a1126889ae40be4e254341e394d40a5723de32...,pt,5.256,n,nan,CV
2,pt-n-0-2-CV.wav,d4c57a15a1126889ae40be4e254341e394d40a5723de32...,pt,5.040,n,nan,CV
3,pt-n-0-3-CV.wav,d4c57a15a1126889ae40be4e254341e394d40a5723de32...,pt,5.736,n,nan,CV
4,pt-n-0-4-CV.wav,d4c57a15a1126889ae40be4e254341e394d40a5723de32...,pt,6.672,n,nan,CV
...,...,...,...,...,...,...,...
34703,pt-m-2178-34703-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.776,m,nan,CV
34704,pt-m-2178-34704-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.944,m,nan,CV
34705,pt-m-2178-34705-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,3.456,m,nan,CV
34706,pt-m-2178-34706-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.176,m,nan,CV


In [14]:
ar = ReadMozilaDataset("../../../DataSets/ExalVoxForge/Mozila_it","../../../DataSets/ExalVoxForge/Mozila_it/clips","../../../DataSets/FinalDataSet/clips","CV")
dfd = ar.Convert(True,"../../../DataSets/FinalDataSet/FinalDataSet.csv")


I_speaker  : 95 / 6492


In [7]:
ar.NewDF

AttributeError: 'ReadMozilaDataset' object has no attribute 'NewDF'

In [9]:
ds = pd.read_csv("../../DataSets/FinalDataSet/FinalDataSet.csv")

In [ ]:
1231 

In [10]:
ds

,Unnamed: 0,filename,speaker,language,length,gender,accent,datasetname
0,0,ar-n-0-0-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.060,n,NaN,CV
1,1,ar-n-0-1-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.176,n,NaN,CV
2,2,ar-n-0-2-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,2.628,n,NaN,CV
3,3,ar-n-0-3-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.168,n,NaN,CV
4,4,ar-n-0-4-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.716,n,NaN,CV
...,...,...,...,...,...,...,...,...
83570,34703,pt-m-2178-34703-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.776,m,NaN,CV
83571,34704,pt-m-2178-34704-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.944,m,NaN,CV
83572,34705,pt-m-2178-34705-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,3.456,m,NaN,CV
83573,34706,pt-m-2178-34706-CV.wav,da70d788a09007d08293d710d0ca147dfb30b3410bcfe0...,pt,4.176,m,NaN,CV


In [3]:
all_files = os.listdir("../../../DataSets/FinalDataSet/clips")

In [5]:
it_CV_files = [i for i in all_files if "CV" in i and "it" in i ]

In [ ]:
DataPath = "D:\GP\YouTube\clips"

In [ ]:
DataSet = []

for lang in os.listdir(DataPath):
    for file in os.listdir(f"{DataPath}/{lang}"):
        gender = file.split("_")[1]
        DataSet.append([file,file,lang,10,gender,None,"YT"])


In [ ]:
df =pd.DataFrame("DataSet")